In [125]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

# データの準備

## データの読み込み

datasetの使い方は[こちら](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html)

In [2]:
wine = load_wine()

In [3]:
inputs = wine.data
targets = wine.target
feature_names = wine.feature_names
target_names  = wine.target_names

In [4]:
print("説明変数：{}".format(feature_names))
print("目的変数：{}".format(target_names))

説明変数：['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']
目的変数：['class_0' 'class_1' 'class_2']


## trainとtestに分ける

train_test_splitの使い方は[こちら](https://docs.pyq.jp/python/machine_learning/tips/train_test_split.html)

In [5]:
# データセットを訓練データとテストデータに分ける
X_train, X_test, y_train, y_test = \
train_test_split(wine.data, wine.target, test_size=0.2)

# CARTの実装

## 特徴量と条件分岐の閾値の決定

### Gini係数の算出

決定木は『不純度』を用いることで分岐条件や特徴量を選択する．
不純度とは，ノード分岐の条件や選択した特徴量(説明変数)によって，ノードに分岐したサンプルのクラスがどれだけ散らばるかを表す指標の1つである．
分岐した際に，クラスがばらけることなく，あるノードに1種類のクラスだけが分類された場合(きれいに分割できた場合)，**そのノードの不純度は0となる**．
不純度を算出する方法は『交差エントロピー』と『Gini係数』があり，CARTの場合は『Gini係数』を用いることで不純度の計算を行う．
Gini係数は，ノード$t$におけるクラス$C_{i}$の割合を$P^{2}(C_{i}|t)$とすると，
$$
1 - \sum_{i=1}^{K}P^{2}(C_{i}|t),
$$
で算出することができる．
Gini係数は，ある条件に対して"Yes"と"No"の，2つ場合に対して求める必要がある(後にGini不純度を計算する必要があるため)．
詳しくは[こちら](https://hktech.hatenablog.com/entry/2018/10/05/004235)．

### Gini不純度の算出

Gini不純度とは，分岐条件毎に算出していた『Gini係数』を用いることで，その分岐条件自体を評価するための指標である．Gini不純度を用いることで，その特徴量における最も分類精度が高い分岐条件を求めることができるので，全特徴量のGini不純度を比較することで，分岐条件だけでなく分類精度が最も高くなる特徴量を選出することも可能となる．ある特徴量における分岐条件のGini不純度は，ノードの集合を$\{\mathrm{yes},\mathrm{no}\}$，1つ前のノードのデータ数を$A$，各ノードのデータ数を$a$とすると，
$$
\sum_{n \in \{\mathrm{yes},\mathrm{no}\}}\frac{a_n}{A} \times \mathrm{Gini}(n),
$$
で算出することができる．ここで$\mathrm{Gini}(n)$とは，yes or no のノードのGini係数である．

### 利得(gain)の算出

利得(gain)とは，分岐前のGini不純度から分岐後のGini不純度を引いた値である．これは分岐後のGini不純度が大きく下がっていれば，それだけその分岐が有効であったことがわかるので，利得が大きければ大きいほどその分岐が重要だということが示される．また各特徴量の分岐の利得を比較すれば，最適な特徴量空間を選出することも可能となる．本来であれば利得を用いて特徴量と閾値を算出すべきだが，今回は利得ではなく各特徴量のGini不純度を比較することでそれらを選出する．

In [113]:
class GiniImpurity(object):
    """
    これは，run()を実行すると"Gini不純度が最も低かった特徴量のindex"と"その時の閾値"を渡してくれるクラスである．
    今回はワインのデータセットを用いているが，入力データやターゲットを変えていただければ
    irisデータセットなどでも適用することが可能である．
    """
    def __init__(self):
        self.inputs = None
        self.targets = None
        self.feature_names = None
        self.target_names = None

        self.gini = None
        self.node = None
        self.yes_num = None
        self.no_num = None
        
        self.impurity = None
        
        self.min_imp = None
        self.threshold = None
        
    
    def calc_gini(self,input_1d):
        """
        ある特徴量に対する全データのGini係数を算出する(2.1参照)．
        
        Parameters
        -------------------------------------------
        input_1d : ndarray
            ある特徴量における現在いるノードの全データである
        -------------------------------------------
        """
        self.gini = np.zeros((len(input_1d),2))
        
        for i, data in enumerate(input_1d):
            branch = input_1d > data
            self.node = np.zeros(2)
            self.yes_num = np.sum(branch)
            self.no_num  = np.sum(np.logical_not(branch))
                        
            for c in range(len(self.target_names)):
                self.node[0] += np.square(np.sum(self.targets[branch]==c) / (self.yes_num + 1e-8))
                self.node[1] += np.square(np.sum(self.targets[np.logical_not(branch)]==c) /
                                          self.no_num + 1e-8)
            for j in range(2):
                self.gini[i,j] = 1 - self.node[j]
    
    
    def calc_impurity(self,input_1d):
        """
        ある特徴量に対する全データのGini係数から，
        分岐の良し悪しを評価するGini不純度を算出し，最小のGini不純度を獲得する(2.2参照)．
        最小のGini不純度を獲得することで，最適な分岐条件と特徴量を決定することが可能となる(2.3参照)．
        
        Parameters
        -------------------------------------------
        input_1d : ndarray
            ある特徴量における現在いるノードの全データである
        -------------------------------------------
        """
        self.calc_gini(input_1d)
        
        self.impurity = np.zeros(len(self.gini))
        self.impurity = self.gini[:,0] * (self.yes_num / (self.yes_num + self.no_num)) + \
                    self.gini[:,1] * (self.no_num  / (self.yes_num + self.no_num))
       
    
    def selc_impurity(self):
        """
        ある特徴量での，Gini不純度が最も小さい時の値とそのindexを返す．
        
        Returns
        -------------------------------------------
        np.min(self.impurity) : int
            ある特徴量における入力された全データの中でGini不純度が最も小さい時の値．
        np.argmin(self.impurity) : int
            ある特徴量における入力された全データの中でGini不純度が最も小さい時のindex．
        -------------------------------------------
        """
        return np.min(self.impurity), np.argmin(self.impurity)
    
    
    def selc_feature(self):
        """
        全特徴量のGini不純度を計算し比較することで，
        クラスが最も綺麗に分かれる特徴量と分岐の閾値を獲得する(2.3参照)．
        
        Returns
        -------------------------------------------
        np.argmin(self.min_imp) : int
            全特徴量の中でGini不純度が最も小さい特徴量のindex．
        self.threshold : float
            全特徴量の中でGini不純度が最も小さい特徴量の閾値．
        -------------------------------------------
        """
        self.min_imp = np.zeros((len(self.inputs[0]),2))
        self.threshold = 0
        
        for i in range(len(self.inputs[0])):
            self.calc_impurity(self.inputs[:,i])
            self.min_imp[i,0], self.min_imp[i,1] = self.selc_impurity()
        
        self.threshold = self.inputs[self.min_imp[np.argmin(self.min_imp[:,0]),1].astype('int64'), \
                                     np.argmin(self.min_imp[:,0])]
        
        return np.argmin(self.min_imp[:,0]), self.threshold
            
        
    
    def run(self,inputs,targets,feature_names,target_names):
        """
        Gini係数とGini不純度の計算を行う．
        最終的に，クラスが最も綺麗に分かれる特徴量と分岐の閾値を獲得する．

        Parameters
        -------------------------------------------
        inputs : ndarray
            全特徴量におけるデータの総数．
            ワインでは，1回目は[178,13]のリストが入るが，
            2回目以降はそのノードにおけるデータの総数となる．
        targets : ndarray
            全特徴量における教師ラベル．
            ワインでは，1回目は178のサイズだが，
            2回目以降はそのノードにおけるデータの総数がサイズとなる．
        feature_names : list
            特徴量の名前.
            ワインでは[13,0]のリストを代入する
        target_names : ndarray
            クラスの名前．
            ワインでは[3,0]のリストを代入する
        -------------------------------------------
        
        Returns
        -------------------------------------------
        self.selc_feature() : int, float
            全特徴量の中で最もGini不純度が小さい特徴量のindexとその特徴量の分岐の閾値．
        -------------------------------------------
        """
        self.inputs = inputs
        self.targets = targets
        self.feature_names = feature_names
        self.target_names = target_names
                
        return self.selc_feature()
        

In [126]:
# 動くか実験
a = GiniImpurity()
b, c = a.run(inputs,targets,feature_names,target_names)
print("最適な特徴量 : {}".format(feature_names[b]))
print("分岐点の閾値 : {}".format(c))

最適な特徴量 : ash
分岐点の閾値 : 1.36


In [127]:
inputs[:,b]

array([2.43, 2.14, 2.67, 2.5 , 2.87, 2.45, 2.45, 2.61, 2.17, 2.27, 2.3 ,
       2.32, 2.41, 2.39, 2.38, 2.7 , 2.72, 2.62, 2.48, 2.56, 2.28, 2.65,
       2.36, 2.52, 2.61, 3.22, 2.62, 2.14, 2.8 , 2.21, 2.7 , 2.36, 2.36,
       2.7 , 2.65, 2.41, 2.84, 2.55, 2.1 , 2.51, 2.31, 2.12, 2.59, 2.29,
       2.1 , 2.44, 2.28, 2.12, 2.4 , 2.27, 2.04, 2.6 , 2.42, 2.68, 2.25,
       2.46, 2.3 , 2.68, 2.5 , 1.36, 2.28, 2.02, 1.92, 2.16, 2.53, 2.56,
       1.7 , 1.92, 2.36, 1.75, 2.21, 2.67, 2.24, 2.6 , 2.3 , 1.92, 1.71,
       2.23, 1.95, 2.4 , 2.  , 2.2 , 2.51, 2.32, 2.58, 2.24, 2.31, 2.62,
       2.46, 2.3 , 2.32, 2.42, 2.26, 2.22, 2.28, 2.2 , 2.74, 1.98, 2.1 ,
       2.21, 1.7 , 1.9 , 2.46, 1.88, 1.98, 2.27, 2.12, 2.28, 1.94, 2.7 ,
       1.82, 2.17, 2.92, 2.5 , 2.5 , 2.2 , 1.99, 2.19, 1.98, 2.  , 2.42,
       3.23, 2.73, 2.13, 2.39, 2.17, 2.29, 2.78, 2.3 , 2.38, 2.32, 2.4 ,
       2.4 , 2.36, 2.25, 2.2 , 2.54, 2.64, 2.19, 2.61, 2.7 , 2.35, 2.72,
       2.35, 2.2 , 2.15, 2.23, 2.48, 2.38, 2.36, 2.

In [117]:
input_1d = inputs[:,b]
gini = np.zeros((len(input_1d),2))

for i, data in enumerate(input_1d):
    branch = input_1d > data
    node = np.zeros(2)
    yes_num = np.sum(branch)
    no_num  = np.sum(np.logical_not(branch))
    if i == 1000:
        print("data : {}".format(data))
        print("branch : {}".format(branch))
        print("yes_num : {}".format(yes_num))
        print("no_num : {}".format(no_num))
        print("targets[branch] : {}".format(targets[branch]))
        print("targets[branch]==0 : {}".format(targets[branch]==0))
        print("np.sum(targets[branch]==0) : {}".format(np.sum(targets[branch]==2)))
                        
    for c in range(len(target_names)):
        node[0] += np.square(np.sum(targets[branch]==c) / (yes_num + 1e-8))
        node[1] += np.square(np.sum(targets[np.logical_not(branch)]==c) / (no_num + 1e-8))
    
    if i == 1000:
        print("node[0] : {}".format(node[0]))
        print("node[1] : {}".format(node[1]))
        
    for j in range(2):
        gini[i,j] = 1 - node[j]

In [118]:
print(gini)

[[6.47491350e-01 6.29256198e-01]
 [6.66018789e-01 3.97502602e-01]
 [6.50283554e-01 6.54235172e-01]
 [6.47189349e-01 6.41975309e-01]
 [4.44444448e-01 6.59461225e-01]
 [6.53808594e-01 6.36503540e-01]
 [6.53808594e-01 6.36503540e-01]
 [6.45328720e-01 6.49209105e-01]
 [6.65641738e-01 4.25925926e-01]
 [6.59699174e-01 5.20540075e-01]
 [6.54538599e-01 5.58577778e-01]
 [6.50637119e-01 5.75990710e-01]
 [6.47762346e-01 6.29583482e-01]
 [6.45956608e-01 6.19800000e-01]
 [6.45937500e-01 6.19533528e-01]
 [6.63265307e-01 6.57644259e-01]
 [6.52777778e-01 6.58658731e-01]
 [6.46666667e-01 6.50474799e-01]
 [6.47321429e-01 6.42837947e-01]
 [6.44672796e-01 6.44609054e-01]
 [6.57684949e-01 5.41322314e-01]
 [6.49600000e-01 6.54790892e-01]
 [6.50892374e-01 6.07041588e-01]
 [6.49729280e-01 6.42509465e-01]
 [6.45328720e-01 6.49209105e-01]
 [1.99999995e-08 6.58942194e-01]
 [6.46666667e-01 6.50474799e-01]
 [6.66018789e-01 3.97502602e-01]
 [6.11111112e-01 6.58396431e-01]
 [6.63108356e-01 4.65973535e-01]
 [6.632653

## 木の構造

# 参考文献

- [sklearn.datasets.load_wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html)
- [train_test_split関数でデータ分割](https://docs.pyq.jp/python/machine_learning/tips/train_test_split.html)
- [Pythonで決定木分類器をフルスクラッチで実装してみた](https://hktech.hatenablog.com/entry/2018/10/05/004235)
- [[入門]初心者の初心者による初心者のための決定木分析](https://qiita.com/3000manJPY/items/ef7495960f472ec14377)

In [27]:
a = np.arange(100)
b = 40 < a

In [44]:
a[np.logical_not(b)]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40])

In [123]:
(60/120)**2

0.25